In [46]:
pip install google-auth google-auth-oauthlib google-api-python-client

Note: you may need to restart the kernel to use updated packages.


In [47]:
import email
import imaplib
import requests
import os
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from email.mime.text import MIMEText
import base64
import html
from bs4 import BeautifulSoup
import re
import openai
import json

In [48]:
# Set up OpenAI API client
from openai import OpenAI

api_key = os.getenv("OPENAI_API_KEY")

# Initialize OpenAI client with API key
client = OpenAI(api_key=api_key)

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": "You are an email reply generator. Based on email trail (or a single email) content, you are expected to create a reply in a professional tone. You use a Gmail api and create an auto-reply with your response."
    },
  ],
  temperature=1,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(response.choices[0].message)

ChatCompletionMessage(content="Subject: Re: Project Update\n\nHi [Client's Name],\n\nThank you for the project update. I appreciate you keeping me in the loop. It's great to hear that everything is progressing smoothly and that we are on track to meet our deadlines.\n\nIf you need any further assistance or have any questions moving forward, please don't hesitate to reach out.\n\nLooking forward to seeing the project come together!\n\nBest regards,\n[Your Name]", role='assistant', function_call=None, tool_calls=None)


In [67]:
# Set up OAuth 2.0 authentication
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

# Load credentials from the token file
def authenticate_gmail():
    if os.path.exists('token.json'):
        with open('token.json', 'r') as token_file:
            credentials_info = json.load(token_file)
        credentials = Credentials.from_authorized_user_info(credentials_info)
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        credentials = flow.run_local_server(port=0)
        with open('token.json', 'w') as token_file:
            token_file.write(credentials.to_json())

    service = build('gmail', 'v1', credentials=credentials)
    return service

def fetch_emails(service):
    # Importing Gmail Data
    results = service.users().messages().list(userId='me', labelIds=['INBOX', 'UNREAD']).execute() #Only unread emails!!
    messages = results.get('messages', [])
    emails = []
    for message in messages:
        try:
            msg = service.users().messages().get(userId='me', id=message['id']).execute()
            headers = msg['payload']['headers']
            sender = next((header['value'] for header in headers if header['name'] == 'From'), None)
            subject = next((header['value'] for header in headers if header['name'] == 'Subject'), None)
            if sender and subject and 'no-reply@tldv.io' not in sender:
                payload = msg['payload']
                parts = payload.get('parts', [payload])
                data = None
                for part in parts:
                    if part['body'].get('data'):
                        data = part['body']['data']
                        break
                if data:
                    try:
                        raw_message = base64.urlsafe_b64decode(data).decode('utf-8')
                    except UnicodeDecodeError:
                        raw_message = base64.urlsafe_b64decode(data).decode('utf-8', 'ignore')  
                    parsed_message = email.message_from_string(raw_message)
                    body = None
                    if parsed_message.is_multipart():
                        for part in parsed_message.walk():
                            content_type = part.get_content_type()
                            if content_type == 'text/html':
                                try:
                                    html_content = part.get_payload(decode=True).decode('utf-8')
                                    soup = BeautifulSoup(html_content, 'html.parser')
                                    body = soup.get_text()
                                except UnicodeDecodeError:
                                    body = part.get_payload(decode=True).decode('utf-8', 'ignore')  
                                break
                    else:
                        try:
                            body = parsed_message.get_payload(decode=True).decode('utf-8')
                        except UnicodeDecodeError:
                            body = parsed_message.get_payload(decode=True).decode('utf-8', 'ignore')  
                    emails.append({'sender': sender, 'subject': subject, 'body': body})
        except KeyError as e:
            print(f"Error processing message: {message['id']}. Missing headers.")
        except Exception as e:
            print(f"Error processing message: {message['id']}. {str(e)}")
    return emails
    
def preprocess_email(body):
    # Clean the email body
    cleaned_body = clean_email_text(body)
    return cleaned_body

def clean_email_text(raw_text):
    
    # Remove email headers (From, To, CC'ed, and Subject)
    header_pattern = r"^(From|To|Cc|Subject):.*$"
    text_without_headers = re.sub(header_pattern, "", raw_text, flags=re.M)
    
    # Remove emojis using unicode
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    text_without_emojis = emoji_pattern.sub(r'', text_without_headers)

    # Strip leading and trailing whitespace
    cleaned_text = text_without_emojis.strip()

    # Remove empty lines
    cleaned_text = re.sub(r'\n\s*\n', '\n', cleaned_text)

    return cleaned_text

def generate_auto_reply(email_content, sender_name):
    prompt = f"Hi {sender_name},\n\nYou received the following email:\n{email_content}\n\nLooking forward to our meeting.\n\nBest regards,\n[Your Name]"
    response = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {
          "role": "system",
          "content": "You are an email reply generator. Based on email trail (or a single email) content, you are expected to create a reply in a professional tone. You use a Gmail api and create an auto-reply with your response. End the email with 'Best Regards, Britney Hydar'"
        },
      ],
      temperature=1,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )

    print(response)  # Add this line to print the response object

    # Extract the generated text from the ChatCompletion object
    auto_reply = response.choices[0].message.content

    auto_reply = auto_reply.replace("[Name]", sender_name)
    
    return auto_reply

# Compose an email message
def create_message(sender_email, subject, body):
    message = MIMEText(body)
    message['to'] = sender_email
    message['subject'] = subject
    return {'raw': base64.urlsafe_b64encode(message.as_string().encode()).decode()}

# Send an email message
def send_message(service, user_id, message):
    try:
        message = (service.users().messages().send(userId=user_id, body=message).execute())
        print('Message Id: %s' % message['id'])
        return message
    except Exception as e:
        print('An error occurred: %s' % e)
        return None

# Send auto-replies
def send_auto_replies(auto_replies, service):
    for reply_data in auto_replies:
        sender_email = reply_data['sender_email']
        subject = reply_data['subject']
        body = reply_data['body']

        # Compose the email
        message = create_message(sender_email, subject, body)

        # Send the email
        send_message(service, 'me', message)
    
def main():
    # Authenticate and build Gmail service
    gmail_service = authenticate_gmail()

    # Fetch emails from Gmail
    emails = fetch_emails(gmail_service)

    # Process fetched emails and fetching the auto-replies
    auto_replies = []
    for email_data in emails:
        # Process the email message as needed
        print("From:", email_data['sender'])
        print("Subject:", email_data['subject'])
        print("Body:", email_data['body'])
        print("---------------------------------------------------")

    # Preprocess email body
        processed_body = preprocess_email(email_data['body'])

        # Generate auto-reply
        sender_name = email_data['sender'].split('<')[0].strip()
        auto_reply = generate_auto_reply(processed_body, sender_name)
        auto_replies.append({'sender_email': email_data['sender'], 'subject': 'Re: ' + email_data['subject'], 'body': auto_reply})

    # Send auto-replies
    send_auto_replies(auto_replies, gmail_service)

if __name__ == "__main__":
    main()

From: Google <no-reply@accounts.google.com>
Subject: Learn more about our updated Terms of Service
Body: taylorwinthrop1027@gmail.com
<https://accounts.google.com/AccountChooser?Email=taylorwinthrop1027%40gmail.com&continue=https%3A%2F%2Fpolicies.google.com%2Fterms%2Fupdate%3Futm_source%3Dtos2024-email%26utm_medium%3Demail>

We\u2019re updating our Terms of Service on May 22, 2024, so we wanted to let
you know ahead of time.

These changes won\u2019t affect the way you use our services
<https://accounts.google.com/AccountChooser?Email=taylorwinthrop1027%40gmail.com&continue=https%3A%2F%2Fpolicies.google.com%2Fterms%2Fservice-specific%3Futm_source%3Dtos2024-email%26utm_medium%3Demail>,
but they should help make it easier for you to understand what to expect
from Google \u2014 and what we expect from you \u2014 as you use our services.

You can see the new terms here
<https://accounts.google.com/AccountChooser?Email=taylorwinthrop1027%40gmail.com&continue=https%3A%2F%2Fpolicies.google.co

KeyboardInterrupt: 